In [1]:
!pip install mysql-connector-python
!pip install pandas


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import mysql.connector
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('superstore.csv', 'superstore')
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='127.0.0.1',
    user='root',
    password='Rudraksh902.',
    database='superstore_db'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = 'C:/Users/vrjav/Downloads/superstore_dataset'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    #df = pd.read_csv(file_path)
    df = pd.read_csv(file_path, encoding='ISO-8859-1') 
    
    # Replace NaN with None to handle SQL NULL
    #df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    #df.columns.str.replace('-','_')

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    conn.commit()

# Close the connection
conn.close()

Processing superstore.csv
NaN values before replacement:
Row ID           0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64



In [3]:
db = mysql.connector.connect(host = "127.0.0.1", username = "root", password = "Rudraksh902.", database = "superstore_db")

cur = db.cursor(buffered=True)

# Total sales by region

In [42]:
cur.execute("USE superstore_db")
query = """ select Region, round(sum(Sales),2) as 'total_sales', round(sum(Profit),2) as 'total_profit' from 
superstore
group by Region """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns = ['region', 'total_sales', 'total_profit'])
df.head()

,region,total_sales,total_profit
0,South,1172427.39,139534.55
1,West,2164386.99,322858.05
2,Central,1500280.65,119541.63
3,East,2029756.53,273717.57


# Top 5 products by Profit

In [41]:
cur.execute("USE superstore_db")
query = """ select `Product Name` as 'prod_name' , round(sum(Profit),2) as 'profit' from superstore
group by `Product Name` 
order by sum(Profit) desc limit 5
 """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns = ['product_name', 'profit'])
df.head(10)

,product_name,profit
0,Canon imageCLASS 2200 Advanced Copier,75599.78
1,Fellowes PB500 Electric Punch Plastic Comb Bin...,23259.12
2,Hewlett Packard LaserJet 3310 Copier,20951.65
3,Canon PC1060 Personal Laser Copier,13712.80
4,Ativa V4110MDD Micro-Cut Shredder,11318.84


# Monthly sales trend (year-wise)

In [40]:
cur.execute("USE superstore_db")
query = """ select YEAR(STR_TO_DATE(`Order Date`, '%m/%d/%Y')) AS year,
    MONTH(STR_TO_DATE(`Order Date`, '%m/%d/%Y')) AS month, round(sum(Sales),2) as 'total_sales' from superstore 
group by YEAR(STR_TO_DATE(`Order Date`, '%m/%d/%Y')), MONTH(STR_TO_DATE(`Order Date`, '%m/%d/%Y'))
order by YEAR(STR_TO_DATE(`Order Date`, '%m/%d/%Y'))asc, MONTH(STR_TO_DATE(`Order Date`, '%m/%d/%Y')) asc
 """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data , columns = ['year', 'month', 'total_sales'])
df.head(10)

,year,month,total_sales
0,2014,1,42635.14
1,2014,2,13159.60
2,2014,3,166908.22
3,2014,4,84730.25
4,2014,5,70931.24
5,2014,6,103699.25
6,2014,7,101393.66
7,2014,8,83422.45
8,2014,9,245051.51
9,2014,10,94301.73


# Top 3 customers by sales

In [39]:
cur.execute("USE superstore_db")
query = """ 
select `Customer Name` as 'customer_name', round(sum(Sales),2) as 'total_sales' from superstore
group by `Customer Name` 
order by sum(Sales) desc limit 3
 """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns = ['customer_name', 'total_sales'])
df.head(10)

,customer_name,total_sales
0,Sean Miller,75129.15
1,Tamara Chand,57122.28
2,Raymond Buch,45352.02


# Average order value (AOV)

In [38]:
cur.execute("USE superstore_db")
query = """ 
select round(sum(Sales)/count(distinct(`Order ID`)),2) as 'avg_order_value' from superstore;
 """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns = ['avg_order_value'])
df.head(10)

,avg_order_value
0,1370.9


# Customer lifetime value (cltv)

In [36]:
cur.execute("USE superstore_db")
query = """ 
select `Customer ID` as 'customer_id', `Customer Name` as 'customer_name', count(`Order ID`) as 'total_orders', 
round(sum(Sales),2) as 'total_sales', round(sum(Profit),2) as 'total_profit'
from superstore
group by `Customer ID`, `Customer Name`
 """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns = ['customer_id', 'customer_name', 'total_orders', 'total_sales', 'total_profit'])
df.head(10)

,customer_id,customer_name,total_orders,total_sales,total_profit
0,CG-12520,Claire Gute,15,3446.34,509.80
1,DV-13045,Darrin Van Huff,27,3358.45,-1281.55
2,SO-20335,Sean O'Donnell,45,7807.73,-243.26
3,BH-11710,Brosina Hoffman,72,18766.05,2408.37
4,AA-10480,Andrew Allen,35,5363.90,1303.74
5,IM-15070,Irene Maddox,63,14791.42,1543.96
6,HP-14815,Harold Pawlan,30,5970.94,1121.59
7,PK-19075,Pete Kriz,73,25452.52,5874.32
8,AG-10270,Alejandro Grove,41,7731.05,2189.58
9,ZD-21925,Zuschuss Donatelli,27,4481.83,747.39


# Profit margin (%)

In [35]:
cur.execute("USE superstore_db")
query = """ 
select round(sum(Profit)*100/sum(Sales),2) as 'profit_margin_percentage' from superstore;
 """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data , columns = ['profit_margin_percentage'])
df.head(10)

,profit_margin_percentage
0,12.46


# Sales by Category and Sub-Category

In [34]:
cur.execute("USE superstore_db")
query = """ 
SELECT Category, `Sub-Category`, round(sum(Sales),2) as 'total_sales'  FROM superstore 
group by Category, `Sub-Category`
 """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns = ['category', 'sub_category', 'total_sales'])
df.head(10)

,category,sub_category,total_sales
0,Furniture,Bookcases,344639.99
1,Furniture,Chairs,985347.31
2,Office Supplies,Labels,37458.94
3,Furniture,Tables,620896.60
4,Office Supplies,Storage,664490.43
5,Furniture,Furnishings,266162.56
6,Office Supplies,Art,81356.38
7,Technology,Phones,989767.20
8,Office Supplies,Binders,606731.18
9,Office Supplies,Appliances,322596.48


# Discount Impact on Profit

In [33]:
cur.execute("USE superstore_db")
query = """ 
select discount_range, avg(Profit) as 'avg_profit' from (SELECT 
case 
when discount between 0.00 and 0.10 then '0-10'
when discount between 0.11 and 0.20 then '11-20'
when discount between 0.21 and 0.30 then '21-30'
when discount between 0.31 and 0.40 then '31-40'
when discount between 0.41 and 0.50 then '41-50'
else '50+' end as 'discount_range', Profit
FROM superstore) t
group by discount_range
order by discount_range
 """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns = ['discount_range', 'avg_profit'])
df.head(10)

,discount_range,avg_profit
0,0-10,67.853236
1,11-20,24.898874
2,21-30,-45.679636
3,31-40,-109.388189
4,41-50,-298.695314
5,50+,-90.653361


# Top 3 States by Sales in Each Region

In [32]:
cur.execute("USE superstore_db")
query = """ 
select Region, State, total_sales from (select *,
row_number() over(partition by Region order by total_sales desc) as 'rn'
from (select Region, State, round(sum(Sales),2) as 'total_sales' from superstore
group by Region, State) t) m
where rn <= 3
 """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns = ['region', 'state', 'total_sales'])
df.head(10)

,region,state,total_sales
0,Central,Texas,509929.72
1,Central,Illinois,239341.49
2,Central,Michigan,228530.81
3,East,New York,931206.17
4,East,Pennsylvania,347935.06
5,East,Ohio,233133.92
6,South,Florida,267824.30
7,South,Virginia,211582.53
8,South,North Carolina,166754.54
9,West,California,1365942.85


# Average Shipping Time

In [31]:
cur.execute("USE superstore_db")
query = """ 
select round(avg(datediff(ship_date,order_date)),2) as 'avg_shipping_time(in_days)'
from (SELECT STR_TO_DATE(`Ship Date`,'%m/%d/%Y') as 'ship_date', STR_TO_DATE(`Order date`,'%m/%d/%Y') as 'order_date'
FROM superstore) t
 """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns = ['avg_shipping_time'])
df.head(10)

,avg_shipping_time
0,3.96


# Repeat vs New Customer Sales

In [43]:
cur.execute("USE superstore_db")
query = """ 
select customer_type, round(sum(Sales),2) as 'total_sales' from (select *, 
case when cnt = 1 then 'new' else 'old' end as 'customer_type'
from (select *, 
dense_rank() over(partition by `Customer Name` order by `Order Date`) as 'rn',
count(*) over(partition by `Customer Name`) as 'cnt' 
from (select `Customer Name`, `Order Date`, round(sum(Sales),2) as 'Sales' from superstore
group by `Customer Name`, `Order Date`) t) m) d
group by customer_type
 """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns = ['customer_type', 'total_sales'])
df.head(10)

,customer_type,total_sales
0,old,6851551.49
1,new,15300.03


# Most Profitable Segment

In [29]:
cur.execute("USE superstore_db")
query = """ 
SELECT Segment, round(sum(Profit),2) as 'total_profit' FROM superstore
group by Segment 
order by total_profit desc limit 1 
 """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data,columns = ['segment', 'total_profit'])
df.head(10)

,segment,total_profit
0,Consumer,400908.19


# Order Count by Shipping Mode

In [28]:
cur.execute("USE superstore_db")
query = """ 
SELECT `Ship Mode`, count(distinct(`Order ID`)) as 'order_count' FROM superstore
group by `Ship Mode`
 """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns = ['ship_mode', 'order_count'])
df.head(10)

,ship_mode,order_count
0,First Class,787
1,Same Day,264
2,Second Class,964
3,Standard Class,2994


# Year-over-Year (YoY) Profit Growth

In [27]:
cur.execute("USE superstore_db")
query = """ 
select order_year as 'year', profit as 'total_profit', (profit-lag_profit)*100/lag_profit as 'YoY_Growth%' from (select *,
lag(profit) over(order by order_year) as 'lag_profit'
from (SELECT year(STR_TO_DATE(`Order Date`,'%m/%d/%Y')) as 'order_year', round(sum(Profit),2) as 'profit' FROM superstore
group by year(STR_TO_DATE(`Order Date`,'%m/%d/%Y')))t) m 
 """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns = ['year', 'total_profit', 'YoY_Growth%'])
df.head(10)

,year,total_profit,YoY_Growth%
0,2014,148132.37,NaN
1,2015,184144.90,24.311047
2,2016,243721.00,32.352837
3,2017,279653.53,14.743305


# Average Profit per Customer by Segment

In [26]:
cur.execute("USE superstore_db")
query = """ 
SELECT Segment, round(sum(Profit)/count(distinct(`Customer Name`)),2) as 'avg_profit_per_customer' FROM superstore
group by Segment
 """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns = ['segment', 'avg_profit_per_customer'])
df.head(10)

,segment,avg_profit_per_customer
0,Consumer,980.22
1,Corporate,1162.39
2,Home Office,1219.05


# Contribution to Profit by Category

In [25]:
cur.execute("USE superstore_db")
query = """ 
select distinct * from (select Category, round((cat_profit),2) as 'total_profit' , round(cat_profit*100/total_profit,2) as 'profit_contribution%'  from (SELECT Category, 
sum(Profit) over (partition by Category) as 'cat_profit',
sum(Profit) over() as 'total_profit'
FROM superstore)t) m
 """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns = ['category', 'total_profit', 'profit_contribution%'])
df.head(10)

,category,total_profit,profit_contribution%
0,Furniture,53883.32,6.30
1,Office Supplies,365471.49,42.71
2,Technology,436296.99,50.99


# Average Discount per Category

In [23]:
cur.execute("USE superstore_db")
query = """ 
select Category, round(avg(Discount),2) as 'avg_discount' from superstore
group by Category
 """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns = ['category', 'avg_discount'])
df.head(10)

,category,avg_discount
0,Furniture,0.17
1,Office Supplies,0.16
2,Technology,0.13


# Customer Profitability Ranking

In [24]:
cur.execute("USE superstore_db")
query = """ 
select *,
dense_rank() over(order by total_profit desc) as 'rank'
from (select `Customer Name`, round(sum(Profit),2) as 'total_profit' from superstore
group by `Customer Name`) t
 """

cur.execute(query)

data = cur.fetchall()

df = pd.DataFrame(data, columns = ['customer_name', 'total_profit', 'rank'])
df.head(10)

,customer_name,total_profit,rank
0,Tamara Chand,26927.13,1
1,Raymond Buch,20928.29,2
2,Sanjit Chand,17272.24,3
3,Hunter Lopez,16867.29,4
4,Adrian Barton,16328.52,5
5,Tom Ashbrook,14111.36,6
6,Christopher Martinez,11699.67,7
7,Keith Dawkins,9115.88,8
8,Andy Reiter,8653.86,9
9,Daniel Raglin,8607.23,10
